# Pokémon battles — LightGBM with 10-fold CV
Notebook che implementa LightGBM con: feature engineering completo, 10-fold CV, Optuna hyperparameter tuning, e submission finale.

# Load data

In [1]:
import json
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# --- Percorsi ---
train_file_path = 'train.jsonl'
test_file_path = 'test.jsonl'

def load_jsonl(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

print('Caricamento dati...')
train_raw = load_jsonl(train_file_path)
test_raw = load_jsonl(test_file_path)
print(f'Train records: {len(train_raw)}, Test records: {len(test_raw)}')

Caricamento dati...
Train records: 10000, Test records: 5000


# Feature Engineering

In [2]:
import math
import numpy as np
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm

# ==============================================================================
# 1. COSTANTI & FUNZIONI BASE
# ==============================================================================

TYPE_CHART = {
    'normal': {'rock': 0.5, 'ghost': 0},
    'fire': {'fire': 0.5, 'water': 0.5, 'grass': 2, 'ice': 2, 'bug': 2, 'rock': 0.5, 'dragon': 0.5},
    'water': {'fire': 2, 'water': 0.5, 'grass': 0.5, 'ground': 2, 'rock': 2, 'dragon': 0.5},
    'grass': {'fire': 0.5, 'water': 2, 'grass': 0.5, 'poison': 0.5, 'ground': 2, 'flying': 0.5, 'bug': 0.5, 'rock': 2, 'dragon': 0.5},
    'electric': {'water': 2, 'grass': 0.5, 'electric': 0.5, 'ground': 0, 'flying': 2, 'dragon': 0.5},
    'ice': {'fire': 0.5, 'water': 0.5, 'grass': 2, 'ground': 2, 'flying': 2, 'dragon': 2},
    'fighting': {'normal': 2, 'ice': 2, 'poison': 0.5, 'flying': 0.5, 'psychic': 0.5, 'bug': 0.5, 'rock': 2, 'ghost': 0},
    'poison': {'grass': 2, 'poison': 0.5, 'ground': 0.5, 'bug': 2, 'rock': 0.5, 'ghost': 0.5},
    'ground': {'fire': 2, 'grass': 0.5, 'electric': 2, 'poison': 2, 'flying': 0, 'bug': 0.5, 'rock': 2},
    'flying': {'grass': 2, 'electric': 0.5, 'fighting': 2, 'bug': 2, 'rock': 0.5},
    'psychic': {'fighting': 2, 'poison': 2, 'psychic': 0.5, 'ghost': 0},
    'bug': {'fire': 0.5, 'grass': 2, 'fighting': 0.5, 'poison': 2, 'flying': 0.5, 'psychic': 2, 'ghost': 0.5},
    'rock': {'fire': 2, 'ice': 2, 'fighting': 0.5, 'ground': 0.5, 'flying': 2, 'bug': 2},
    'ghost': {'normal': 0, 'psychic': 0, 'ghost': 2},
    'dragon': {'dragon': 2}
}

def get_effectiveness(attack_type: str, defense_types: list) -> float:
    if not attack_type or not defense_types: return 1.0
    eff = 1.0
    for d in defense_types: eff *= TYPE_CHART.get(attack_type, {}).get(d, 1.0)
    return eff

def _entropy(counter: Counter) -> float:
    total = sum(counter.values())
    if total == 0: return 0.0
    ent = 0.0
    for v in counter.values():
        p = v / total
        if p > 0: ent -= p * math.log(p, 2)
    return ent

# ==============================================================================
# 2. ESTRATTORI FEATURE ORIGINALI (CONSERVATI)
# ==============================================================================

def calculate_type_advantage(team1: list, team2_lead: dict) -> dict:
    out = {'p1_vs_lead_avg_effectiveness': 0.0, 'p1_vs_lead_max_effectiveness': 0.0, 'p1_super_effective_options': 0}
    if not team1 or not team2_lead: return out
    lead_types = [t.lower() for t in team2_lead.get('types', [])]
    if not lead_types: return out
    effs = []
    for p in team1:
        p_types = [t.lower() for t in p.get('types', [])]
        max_eff = 0.0
        for pt in p_types: max_eff = max(max_eff, get_effectiveness(pt, lead_types))
        effs.append(max_eff)
    if not effs: return out
    out['p1_vs_lead_avg_effectiveness'] = float(np.mean(effs))
    out['p1_vs_lead_max_effectiveness'] = float(np.max(effs))
    out['p1_super_effective_options'] = int(sum(1 for e in effs if e >= 2))
    return out

def team_aggregate_features(team: list, prefix: str = 'p1_') -> dict:
    stats = ['base_hp','base_atk','base_def','base_spa','base_spd','base_spe']
    out = {}
    vals = {s: [] for s in stats}
    levels = []; types_counter = Counter(); names = []
    for p in team:
        names.append(p.get('name',''))
        for s in stats: vals[s].append(p.get(s, 0))
        levels.append(p.get('level', 0))
        for t in p.get('types', []): types_counter[t.lower()] += 1
    for s in stats:
        arr = np.array(vals[s], dtype=float)
        out[f'{prefix}{s}_sum'] = float(arr.sum())
        out[f'{prefix}{s}_mean'] = float(arr.mean())
        out[f'{prefix}{s}_max'] = float(arr.max())
        out[f'{prefix}{s}_min'] = float(arr.min())
        out[f'{prefix}{s}_std'] = float(arr.std())
    level_arr = np.array(levels, dtype=float)
    out[f'{prefix}level_mean'] = float(level_arr.mean()) if level_arr.size else 0.0
    out[f'{prefix}level_sum'] = float(level_arr.sum()) if level_arr.size else 0.0
    out[f'{prefix}n_unique_types'] = int(len(types_counter))
    for t in ['normal','fire','water','electric','grass','psychic','ice','dragon','rock','ground','flying']:
        out[f'{prefix}type_{t}_count'] = int(types_counter.get(t, 0))
    out[f'{prefix}lead_name'] = names[0] if names else ''
    out[f'{prefix}n_unique_names'] = int(len(set(names)))
    out[f'{prefix}type_entropy'] = float(_entropy(types_counter))
    spe_arr = np.array(vals['base_spe'], dtype=float)
    out[f'{prefix}spe_p25'] = float(np.percentile(spe_arr, 25)) if spe_arr.size else 0.0
    out[f'{prefix}spe_p50'] = float(np.percentile(spe_arr, 50)) if spe_arr.size else 0.0
    out[f'{prefix}spe_p75'] = float(np.percentile(spe_arr, 75)) if spe_arr.size else 0.0
    return out

def lead_vs_lead_features(p1_lead: dict, p2_lead: dict) -> dict:
    out = {}
    for s in ['base_hp','base_atk','base_def','base_spa','base_spd','base_spe']:
        out[f'lead_diff_{s}'] = float(p1_lead.get(s,0) - p2_lead.get(s,0))
    out['lead_speed_advantage'] = float(p1_lead.get('base_spe',0) - p2_lead.get('base_spe',0))
    p1_types = [t.lower() for t in p1_lead.get('types', [])]
    p2_types = [t.lower() for t in p2_lead.get('types', [])]
    max_eff = 0.0
    for pt in p1_types: max_eff = max(max_eff, get_effectiveness(pt, p2_types))
    out['lead_p1_vs_p2_effectiveness'] = float(max_eff)
    return out

def lead_aggregate_features(pokemon: dict, prefix: str = 'p2_lead_') -> dict:
    out = {}
    for s in ['base_hp','base_atk','base_def','base_spa','base_spd','base_spe']:
        out[f'{prefix}{s}'] = float(pokemon.get(s,0))
    out[f'{prefix}level'] = int(pokemon.get('level',0))
    types = [x.lower() for x in pokemon.get('types', [])]
    for t in ['normal','fire','water','electric','grass','psychic','ice','dragon','rock','ground','flying']:
        out[f'{prefix}type_{t}'] = int(t in types)
    out[f'{prefix}name'] = pokemon.get('name','')
    out[f'{prefix}n_unique_types'] = int(len(set(types)))
    return out

def quick_boost_features_v2(record: dict) -> dict:
    out = {}
    p1_team = record.get('p1_team_details', [])
    p2_lead = record.get('p2_lead_details', {})
    timeline = record.get('battle_timeline', [])
    if not p1_team: return out
    
    p2_lead_spe = p2_lead.get('base_spe', 0)
    out['p1_faster_than_lead_count'] = sum(1 for p in p1_team if p.get('base_spe', 0) > p2_lead_spe)
    out['p1_slower_than_lead_count'] = sum(1 for p in p1_team if p.get('base_spe', 0) <= p2_lead_spe)
    out['p1_speed_control_ratio'] = out['p1_faster_than_lead_count'] / max(1, len(p1_team))
    
    p1_avg_bulk = np.mean([p.get('base_hp', 0)*(p.get('base_def', 0)+p.get('base_spd', 0)) for p in p1_team])
    p2_lead_bulk = p2_lead.get('base_hp', 1)*(p2_lead.get('base_def', 1)+p2_lead.get('base_spd', 1))
    out['p1_avg_bulk_vs_lead'] = p1_avg_bulk / max(p2_lead_bulk, 1)
    
    p1_total_atk = sum(p.get('base_atk', 0) + p.get('base_spa', 0) for p in p1_team)
    p2_lead_offense = p2_lead.get('base_atk', 0) + p2_lead.get('base_spa', 0)
    out['p1_total_offense'] = p1_total_atk
    out['p1_offense_advantage'] = p1_total_atk / max(p2_lead_offense, 1)
    
    p2_lead_types = [t.lower() for t in p2_lead.get('types', [])]
    if p2_lead_types:
        cov = []
        for p in p1_team:
            p_types = [t.lower() for t in p.get('types', [])]
            cov.append(max([get_effectiveness(pt, p2_lead_types) for pt in p_types] or [1.0]))
        out['p1_avg_effectiveness_vs_lead'] = float(np.mean(cov))
        out['p1_max_effectiveness_vs_lead'] = float(np.max(cov))
        out['p1_se_count_vs_lead'] = sum(1 for s in cov if s >= 2.0)
        out['p1_weak_count_vs_lead'] = sum(1 for s in cov if s <= 0.5)
        
    if timeline:
        first_p1_ko = False; first_p2_ko = False
        for turn in timeline[:30]:
            if not first_p2_ko and turn.get('p2_pokemon_state', {}).get('fainted'):
                first_p1_ko = True; out['p1_first_blood'] = 1; out['p1_first_blood_turn'] = turn.get('turn', 0); break
            if not first_p1_ko and turn.get('p1_pokemon_state', {}).get('fainted'):
                first_p2_ko = True; out['p1_first_blood'] = 0; out['p1_first_blood_turn'] = turn.get('turn', 0); break
        if not first_p1_ko and not first_p2_ko:
            out['p1_first_blood'] = -1; out['p1_first_blood_turn'] = 0
            
    p1_avg_level = np.mean([p.get('level', 50) for p in p1_team])
    out['p1_avg_level_advantage'] = p1_avg_level - p2_lead.get('level', 50)
    
    p1_prods = [(p.get('base_hp',1)*p.get('base_atk',1)*p.get('base_def',1)*p.get('base_spa',1)*p.get('base_spd',1)*p.get('base_spe',1)) for p in p1_team]
    out['p1_avg_stat_product'] = float(np.mean(p1_prods))
    out['p1_max_stat_product'] = float(np.max(p1_prods))
    p2_prod = p2_lead.get('base_hp',1)*p2_lead.get('base_atk',1)*p2_lead.get('base_def',1)*p2_lead.get('base_spa',1)*p2_lead.get('base_spd',1)*p2_lead.get('base_spe',1)
    out['p1_stat_product_advantage'] = out['p1_avg_stat_product'] / max(p2_prod, 1)
    return out

def summary_from_timeline(timeline: list, p1_team: list) -> dict:
    out = {}
    if not timeline: return {'tl_p1_moves':0,'tl_p2_moves':0,'tl_p1_est_damage':0.0,'tl_p2_est_damage':0.0,'damage_diff':0.0}
    p1_moves = p2_moves = 0; p1_damage = p2_damage = 0.0
    p1_last_active = p2_last_active = ''; p1_fainted = p2_fainted = 0
    p1_fainted_names = set(); p2_fainted_names = set()
    last_p1_hp = {}; last_p2_hp = {}
    p1_comeback_kos = p2_comeback_kos = 0
    p1_statuses = Counter(); p2_statuses = Counter()
    p1_poke_statuses = {}; p2_poke_statuses = {}
    p1_move_types = Counter(); p2_move_types = Counter()
    p1_dmg_first2 = p2_dmg_first2 = 0.0
    p1_dmg_by_turn = {}; p2_dmg_by_turn = {}; seen_turns = set()
    first_ko_p1_taken = None; first_ko_p1_inflicted = None
    p1_kos_early = p1_kos_late = p2_kos_early = p2_kos_late = 0

    for turn in timeline[:30]:
        prev_p1_f, prev_p2_f = p1_fainted, p2_fainted
        p1s = turn.get('p1_pokemon_state',{}) or {}; p2s = turn.get('p2_pokemon_state',{}) or {}
        tnum = turn.get('turn', len(seen_turns) + 1); seen_turns.add(tnum)

        if p1s.get('name'): p1_last_active = p1s.get('name')
        if p2s.get('name'): p2_last_active = p2s.get('name')

        if p1s.get('fainted') and p1s.get('name') not in p1_fainted_names:
            p1_fainted += 1; p1_fainted_names.add(p1s.get('name'))
            if first_ko_p1_taken is None: first_ko_p1_taken = tnum
            if tnum <= 10: p2_kos_early += 1
            else: p2_kos_late += 1
        if p2s.get('fainted') and p2s.get('name') not in p2_fainted_names:
            p2_fainted += 1; p2_fainted_names.add(p2s.get('name'))
            if first_ko_p1_inflicted is None: first_ko_p1_inflicted = tnum
            if tnum <= 10: p1_kos_early += 1
            else: p1_kos_late += 1

        if p2s.get('name') and p2s.get('hp_pct') is not None:
            prev = last_p2_hp.get(p2s['name'])
            if prev is not None:
                delta = max(0.0, prev - p2s['hp_pct'])
                p1_damage += delta; p1_dmg_by_turn[tnum] = p1_dmg_by_turn.get(tnum,0.0)+delta
                if tnum <= 2: p1_dmg_first2 += delta
            last_p2_hp[p2s['name']] = p2s['hp_pct']

        if p1s.get('name') and p1s.get('hp_pct') is not None:
            prev = last_p1_hp.get(p1s['name'])
            if prev is not None:
                delta = max(0.0, prev - p1s['hp_pct'])
                p2_damage += delta; p2_dmg_by_turn[tnum] = p2_dmg_by_turn.get(tnum,0.0)+delta
                if tnum <= 2: p2_dmg_first2 += delta
            last_p1_hp[p1s['name']] = p1s['hp_pct']

        dmg_diff = p1_damage - p2_damage
        if p2_fainted > prev_p2_f and dmg_diff < -1.0: p1_comeback_kos += 1
        if p1_fainted > prev_p1_f and dmg_diff > 1.0: p2_comeback_kos += 1

        if p2s.get('name') and p2s.get('status') and p2_poke_statuses.get(p2s['name']) != p2s['status']:
            p1_statuses[p2s['status']] += 1; p2_poke_statuses[p2s['name']] = p2s['status']
        if p1s.get('name') and p1s.get('status') and p1_poke_statuses.get(p1s['name']) != p1s['status']:
            p2_statuses[p1s['status']] += 1; p1_poke_statuses[p1s['name']] = p1s['status']

        if turn.get('p1_move_details'):
            p1_moves += 1; mtype = (turn['p1_move_details'].get('type') or '').lower()
            if mtype: p1_move_types[mtype] += 1
        if turn.get('p2_move_details'):
            p2_moves += 1; mtype = (turn['p2_move_details'].get('type') or '').lower()
            if mtype: p2_move_types[mtype] += 1

    out['tl_p1_moves'] = p1_moves; out['tl_p2_moves'] = p2_moves
    out['tl_p1_est_damage'] = float(p1_damage); out['tl_p2_est_damage'] = float(p2_damage)
    out['tl_p1_fainted'] = p1_fainted; out['tl_p2_fainted'] = p2_fainted
    turns = max(1, len(seen_turns))
    out['tl_p1_fainted_rate'] = p1_fainted/turns; out['tl_p2_fainted_rate'] = p2_fainted/turns
    out['damage_diff'] = float(p1_damage - p2_damage); out['fainted_diff'] = int(p1_fainted - p2_fainted)
    out['tl_p1_last_hp'] = float(last_p1_hp.get(p1_last_active, 0) if p1_last_active else 0)
    out['tl_p2_last_hp'] = float(last_p2_hp.get(p2_last_active, 0) if p2_last_active else 0)
    out['tl_p1_last_active'] = p1_last_active; out['tl_p2_last_active'] = p2_last_active
    
    if p1_team:
        p1_hp_sum = sum(p.get('base_hp',0) for p in p1_team)
        p1_def_avg = np.mean([p.get('base_def',0)+p.get('base_spd',0) for p in p1_team])
        out['tl_p2_damage_vs_p1_hp_pool'] = p2_damage / (p1_hp_sum + 1e-6)
        out['tl_p1_defensive_endurance'] = p1_def_avg / (p2_damage + 1e-6)
        
    out['tl_p1_comeback_kos'] = p1_comeback_kos; out['tl_p2_comeback_kos'] = p2_comeback_kos
    out['tl_comeback_kos_diff'] = p1_comeback_kos - p2_comeback_kos

    for s in ['brn','par','slp','frz','psn','tox']:
        out[f'tl_p1_inflicted_{s}_count'] = p1_statuses[s]
        out[f'tl_p2_inflicted_{s}_count'] = p2_statuses[s]
        out[f'tl_inflicted_{s}_diff'] = p1_statuses[s] - p2_statuses[s]
        out[f'tl_p1_inflicted_{s}_rate'] = p1_statuses[s]/turns
        out[f'tl_p2_inflicted_{s}_rate'] = p2_statuses[s]/turns
        out[f'tl_inflicted_{s}_rate_diff'] = (p1_statuses[s]-p2_statuses[s])/turns

    for mt in ['normal','fire','water','electric','grass','psychic','ice','dragon','rock','ground','flying','ghost','bug','poison','fighting']:
        out[f'tl_p1_move_type_{mt}_count'] = p1_move_types[mt]
        out[f'tl_p2_move_type_{mt}_count'] = p2_move_types[mt]
        out[f'tl_move_type_{mt}_count_diff'] = p1_move_types[mt] - p2_move_types[mt]

    out['tl_p1_damage_first2'] = p1_dmg_first2; out['tl_p2_damage_first2'] = p2_dmg_first2
    out['tl_first2_damage_diff'] = p1_dmg_first2 - p2_dmg_first2
    out['tl_turns_count'] = turns
    out['tl_p1_moves_rate'] = p1_moves/turns; out['tl_p2_moves_rate'] = p2_moves/turns
    out['tl_p1_damage_per_turn'] = p1_damage/turns; out['tl_p2_damage_per_turn'] = p2_damage/turns
    out['tl_damage_rate_diff'] = out['tl_p1_damage_per_turn'] - out['tl_p2_damage_per_turn']

    recent = sorted(seen_turns)[-5:] if seen_turns else []
    p1_last5 = sum(p1_dmg_by_turn.get(t,0) for t in recent)
    p2_last5 = sum(p2_dmg_by_turn.get(t,0) for t in recent)
    out['tl_p1_damage_last5'] = p1_last5; out['tl_p2_damage_last5'] = p2_last5
    out['tl_last5_damage_diff'] = p1_last5 - p2_last5
    out['tl_p1_last5_damage_ratio'] = p1_last5/(p1_damage+1e-6)
    out['tl_p2_last5_damage_ratio'] = p2_last5/(p2_damage+1e-6)
    out['tl_last5_damage_ratio_diff'] = out['tl_p1_last5_damage_ratio'] - out['tl_p2_last5_damage_ratio']

    if seen_turns:
        ts = sorted(seen_turns); w = np.linspace(1.0, 2.0, num=len(ts)); w = w/w.sum()
        adv = [(p1_dmg_by_turn.get(t,0)-p2_dmg_by_turn.get(t,0)) for t in ts]
        out['tl_weighted_damage_diff'] = float(np.dot(w, adv))
        cum = 0.0; signs = []
        for t in ts:
            cum += (p1_dmg_by_turn.get(t,0)-p2_dmg_by_turn.get(t,0))
            s = 1 if cum > 1e-9 else (-1 if cum < -1e-9 else 0)
            if s != 0 and (not signs or signs[-1] != s): signs.append(s)
        out['tl_damage_adv_sign_flips'] = max(0, len(signs)-1)
        out['tl_comeback_flag'] = 1 if (len(signs)>=2 and signs[0]!=signs[-1]) else 0
    else:
        out['tl_weighted_damage_diff'] = 0.0; out['tl_damage_adv_sign_flips'] = 0; out['tl_comeback_flag'] = 0

    out['tl_first_ko_turn_p1_inflicted'] = int(first_ko_p1_inflicted or 0)
    out['tl_first_ko_turn_p1_taken'] = int(first_ko_p1_taken or 0)
    out['tl_first_ko_turn_diff'] = out['tl_first_ko_turn_p1_inflicted'] - out['tl_first_ko_turn_p1_taken']
    out['tl_kos_early_p1'] = p1_kos_early; out['tl_kos_late_p1'] = p1_kos_late
    out['tl_kos_early_p2'] = p2_kos_early; out['tl_kos_late_p2'] = p2_kos_late
    return out

def extract_move_coverage_from_timeline(timeline: list, prefix: str = 'p1_') -> dict:
    out = {}
    m_types = set(); m_cats = Counter(); unique_m = set(); stab = 0
    for turn in timeline[:30]:
        md = turn.get(f'{prefix[:-1]}_move_details')
        ps = turn.get(f'{prefix[:-1]}_pokemon_state', {})
        if not md: continue
        if md.get('name'): unique_m.add(md['name'])
        if t := (md.get('type') or '').lower(): m_types.add(t)
        if c := md.get('category'): m_cats[c] += 1
        if t in [pt.lower() for pt in ps.get('types', [])]: stab += 1
    
    out[f'{prefix}tl_unique_move_types'] = len(m_types)
    out[f'{prefix}tl_unique_moves_used'] = len(unique_m)
    out[f'{prefix}tl_stab_moves'] = stab
    out[f'{prefix}tl_physical_moves'] = m_cats['physical']
    out[f'{prefix}tl_special_moves'] = m_cats['special']
    out[f'{prefix}tl_status_moves'] = m_cats['status']
    out[f'{prefix}tl_coverage_score'] = len(m_types)/max(1, len(unique_m))
    tot = sum(m_cats.values())
    out[f'{prefix}tl_offensive_ratio'] = (m_cats['physical']+m_cats['special'])/tot if tot else 0.0
    out[f'{prefix}tl_status_ratio'] = m_cats['status']/tot if tot else 0.0
    return out

def ability_features(team: list, prefix: str) -> dict:
    imm = {'levitate':0,'volt_absorb':0,'water_absorb':0,'flash_fire':0}
    drop = {'intimidate':0}; weather = {'drought':0,'drizzle':0,'sand_stream':0}
    for p in team:
        a = (p.get('ability','') or '').lower().replace(' ','_')
        if a in imm: imm[a]+=1
        if a in drop: drop[a]+=1
        if a in weather: weather[a]+=1
    out = {}
    for k,v in imm.items(): out[f'{prefix}ability_{k}_count'] = v
    for k,v in drop.items(): out[f'{prefix}ability_{k}_count'] = v
    for k,v in weather.items(): out[f'{prefix}ability_{k}_count'] = v
    out[f'{prefix}total_immunity_abilities'] = sum(imm.values())
    out[f'{prefix}total_stat_drop_abilities'] = sum(drop.values())
    return out

def momentum_features(timeline: list) -> dict:
    out = {}
    if not timeline: return out
    advs = []; cum = 0.0
    for turn in timeline[:30]:
        p1h = turn.get('p1_pokemon_state', {}).get('hp_pct', 100)
        p2h = turn.get('p2_pokemon_state', {}).get('hp_pct', 100)
        cum += (p1h - p2h); advs.append(cum)
    if advs:
        slope, intercept = np.polyfit(np.arange(len(advs)), advs, 1)
        out['p1_momentum_slope'] = float(slope)
        out['p1_momentum_intercept'] = float(intercept)
        out['p1_final_advantage'] = float(advs[-1])
        out['p1_advantage_volatility'] = float(np.std(advs))
        out['p1_max_advantage'] = float(np.max(advs))
        out['p1_min_advantage'] = float(np.min(advs))
        out['p1_advantage_range'] = out['p1_max_advantage'] - out['p1_min_advantage']
    return out

def extract_opponent_team_from_timeline(timeline: list, p1_team: list) -> dict:
    out = {}
    seen = set(); types = []
    for turn in timeline[:30]:
        p2s = turn.get('p2_pokemon_state', {})
        if p2s.get('name') and p2s['name'] not in seen:
            seen.add(p2s['name'])
            types.extend([t.lower() for t in p2s.get('types', [])])
    
    out['p2_tl_unique_pokemon_seen'] = len(seen)
    out['p2_tl_switches_count'] = max(0, len(seen) - 1)
    out['p2_tl_unique_types_seen'] = len(set(types))
    out['p2_tl_type_entropy'] = _entropy(Counter(types))
    
    advs = 0
    if types and p1_team:
        for p1 in p1_team:
            for t1 in [t.lower() for t in p1.get('types', [])]:
                for t2 in set(types):
                    if get_effectiveness(t1, [t2]) >= 2.0: advs += 1
    out['p1_vs_p2_tl_type_advantages'] = advs
    out['p1_vs_p2_tl_type_advantages_per_poke'] = advs / max(1, len(p1_team))
    out['p2_tl_switch_rate'] = len(seen) / max(1, len(timeline[:30]))
    return out

# ==============================================================================
# 3. NUOVE FEATURE AVANZATE (DATA-DRIVEN, NO META)
# ==============================================================================

def extract_information_advantage(timeline: list) -> dict:
    p1_rev = set(); p2_rev = set(); reveal_turns_p2 = []
    for turn in timeline[:30]:
        t = turn.get('turn', 0)
        if n1 := turn.get('p1_pokemon_state', {}).get('name'): p1_rev.add(n1)
        if n2 := turn.get('p2_pokemon_state', {}).get('name'):
            if n2 not in p2_rev: p2_rev.add(n2); reveal_turns_p2.append(t)
    return {
        'tl_p1_revealed_count': len(p1_rev),
        'tl_p2_revealed_count': len(p2_rev),
        'tl_info_advantage': len(p2_rev) - len(p1_rev),
        'tl_p2_avg_reveal_turn': float(np.mean(reveal_turns_p2)) if reveal_turns_p2 else 30.0
    }

def extract_advanced_momentum(timeline: list) -> dict:
    p1_immune = 0; p2_forced = 0
    for i, turn in enumerate(timeline[:30]):
        if i == 0: continue
        prev = timeline[i-1]
        c1 = turn.get('p1_pokemon_state', {}).get('name')
        p1 = prev.get('p1_pokemon_state', {}).get('name')
        if c1 != p1 and not prev.get('p1_pokemon_state', {}).get('fainted'):
            m2_type = (turn.get('p2_move_details') or {}).get('type', '').lower()
            p1_types = [t.lower() for t in turn.get('p1_pokemon_state', {}).get('types', [])]
            if m2_type and p1_types and get_effectiveness(m2_type, p1_types) == 0.0:
                p1_immune += 1
        c2 = turn.get('p2_pokemon_state', {}).get('name')
        p2 = prev.get('p2_pokemon_state', {}).get('name')
        if c2 != p2 and not prev.get('p2_pokemon_state', {}).get('fainted'):
            if prev.get('p2_pokemon_state', {}).get('hp_pct', 1.0) < 0.50:
                p2_forced += 1
    return {'tl_p1_immune_switches': p1_immune, 'tl_p2_forced_switches': p2_forced}

def extract_gamestate_snapshots(timeline: list) -> dict:
    turns_lead = 0; hp_diff_10 = 0.0; hp_diff_20 = 0.0; hp_diff_30 = 0.0
    for i, turn in enumerate(timeline[:30]):
        t = i + 1
        h1 = turn.get('p1_pokemon_state', {}).get('hp_pct', 0)
        h2 = turn.get('p2_pokemon_state', {}).get('hp_pct', 0)
        if h1 > h2: turns_lead += 1
        if t == 10: hp_diff_10 = h1 - h2
        if t == 20: hp_diff_20 = h1 - h2
        hp_diff_30 = h1 - h2 # Ultimo stato disponibile
    return {
        'tl_turns_with_hp_lead': turns_lead,
        'tl_hp_diff_turn_10': float(hp_diff_10),
        'tl_hp_diff_turn_20': float(hp_diff_20),
        'tl_hp_diff_end': float(hp_diff_30)
    }

def extract_observed_mechanics(timeline: list) -> dict:
    p1_heals = 0; p2_heals = 0; p1_frz = 0; p2_frz = 0
    for i, turn in enumerate(timeline[:30]):
        if i == 0: continue
        prev = timeline[i-1]
        p1s = turn.get('p1_pokemon_state', {}); p1s_prev = prev.get('p1_pokemon_state', {})
        p2s = turn.get('p2_pokemon_state', {}); p2s_prev = prev.get('p2_pokemon_state', {})
        if p1s.get('name') == p1s_prev.get('name'):
             if p1s.get('hp_pct', 0) > p1s_prev.get('hp_pct', 0): p1_heals += 1
        if p2s.get('name') == p2s_prev.get('name'):
             if p2s.get('hp_pct', 0) > p2s_prev.get('hp_pct', 0): p2_heals += 1
        if p1s.get('status') == 'frz': p1_frz = 1
        if p2s.get('status') == 'frz': p2_frz = 1
    return {'tl_heal_diff': p1_heals - p2_heals, 'tl_freeze_adv': p2_frz - p1_frz}

# ==============================================================================
# 4. MASTER FUNCTION V3 (COMPLETA)
# ==============================================================================

def prepare_record_features_COMPLETE(record: dict, max_turns: int = 30) -> dict:
    out = {'battle_id': record.get('battle_id')}
    if 'player_won' in record: out['player_won'] = int(bool(record['player_won']))
    
    p1_team = record.get('p1_team_details', [])
    p2_lead = record.get('p2_lead_details', {})
    p1_lead = p1_team[0] if p1_team else {}
    tl = record.get('battle_timeline', [])[:max_turns]
    
    # --- VECCHIE FEATURE (MANTENUTE) ---
    out.update(team_aggregate_features(p1_team, 'p1_'))
    out.update(lead_aggregate_features(p2_lead, 'p2_lead_'))
    out.update(ability_features(p1_team, 'p1_'))
    out.update(lead_vs_lead_features(p1_lead, p2_lead))
    out.update(ability_features([p2_lead], 'p2_lead_'))
    out['p1_intimidate_vs_lead'] = int(out.get('p1_ability_intimidate_count',0) > 0)
    out.update(summary_from_timeline(tl, p1_team))
    out.update(extract_move_coverage_from_timeline(tl, 'p1_'))
    out.update(extract_move_coverage_from_timeline(tl, 'p2_'))
    out.update(extract_opponent_team_from_timeline(tl, p1_team))
    out.update(quick_boost_features_v2(record))
    out.update(momentum_features(tl))
    
    out['team_hp_sum_minus_p2lead_hp'] = out.get('p1_base_hp_sum',0) - out.get('p2_lead_base_hp',0)
    out['team_spa_mean_minus_p2spa'] = out.get('p1_base_spa_mean',0) - out.get('p2_lead_base_spa',0)
    out['speed_advantage'] = out.get('p1_base_spe_sum',0) - out.get('p2_lead_base_spe',0)
    out['n_unique_types_diff'] = out.get('p1_n_unique_types',0) - out.get('p2_lead_n_unique_types',1)
    p1m = max(out.get('tl_p1_moves',1),1); p2m = max(out.get('tl_p2_moves',1),1)
    out['damage_per_turn_diff'] = (out.get('tl_p1_est_damage',0)/p1m) - (out.get('tl_p2_est_damage',0)/p2m)
    out['last_pair'] = f"{out.get('tl_p1_last_active','')}_VS_{out.get('tl_p2_last_active','')}"
    
    out.update(calculate_type_advantage(p1_team, p2_lead))
    p2_bulk = out.get('p2_lead_base_def',1) + out.get('p2_lead_base_spd',1)
    out['p1_se_options_vs_lead_bulk'] = out.get('p1_super_effective_options',0) / (p2_bulk + 1e-6)
    
    if p2_team := record.get('p2_team_details', []):
        out.update(team_aggregate_features(p2_team, 'p2_'))
        out['team_hp_sum_diff'] = out.get('p1_base_hp_sum',0) - out.get('p2_base_hp_sum',0)
        out['team_spa_mean_diff'] = out.get('p1_base_spa_mean',0) - out.get('p2_base_spa_mean',0)
        out['team_spe_mean_diff'] = out.get('p1_base_spe_mean',0) - out.get('p2_base_spe_mean',0)
        out['n_unique_types_team_diff'] = out.get('p1_n_unique_types',0) - out.get('p2_n_unique_types',0)
        
    # --- NUOVE FEATURE (AGGIUNTE) ---
    if tl:
        out.update(extract_information_advantage(tl))
        out.update(extract_advanced_momentum(tl))
        out.update(extract_gamestate_snapshots(tl))
        out.update(extract_observed_mechanics(tl))
    else:
        # Defaults per record senza timeline
        out.update({
            'tl_p1_revealed_count': 1, 'tl_p2_revealed_count': 1, 'tl_info_advantage': 0,
            'tl_p2_avg_reveal_turn': 30.0, 'tl_p1_immune_switches': 0, 'tl_p2_forced_switches': 0,
            'tl_turns_with_hp_lead': 0, 'tl_hp_diff_turn_10': 0.0, 'tl_hp_diff_turn_20': 0.0,
            'tl_hp_diff_end': 0.0, 'tl_heal_diff': 0, 'tl_freeze_adv': 0
        })
        
    return out

def create_features_from_raw(data: list) -> pd.DataFrame:
    rows = []
    for b in tqdm(data, desc='FE Complete'):
        try:
            # Usa la funzione master completa
            feat = prepare_record_features_COMPLETE(b)
            if 'battle_id' not in feat: feat['battle_id'] = b.get('battle_id')
            rows.append(feat)
        except Exception as e:
            rows.append({'battle_id': b.get('battle_id'), 'error': 1})
    df = pd.DataFrame(rows)
    if 'player_won' in df.columns: df['player_won'] = df['player_won'].astype(int)
    return df.fillna(0)

# Esecuzione
train_df = create_features_from_raw(train_raw)
test_df = create_features_from_raw(test_raw)
print('Feature shape train/test:', train_df.shape, test_df.shape)
display(train_df.head())

FE Complete:   0%|          | 0/10000 [00:00<?, ?it/s]

FE Complete:   0%|          | 0/5000 [00:00<?, ?it/s]

Feature shape train/test: (10000, 296) (5000, 295)


,battle_id,player_won,p1_base_hp_sum,p1_base_hp_mean,p1_base_hp_max,p1_base_hp_min,p1_base_hp_std,p1_base_atk_sum,p1_base_atk_mean,p1_base_atk_max,...,tl_info_advantage,tl_p2_avg_reveal_turn,tl_p1_immune_switches,tl_p2_forced_switches,tl_turns_with_hp_lead,tl_hp_diff_turn_10,tl_hp_diff_turn_20,tl_hp_diff_end,tl_heal_diff,tl_freeze_adv
0,0,1,695.0,115.833333,250.0,55.0,69.367179,435.0,72.500000,110.0,...,0,8.500000,0,3,12,-0.58891,-0.201018,0.279549,0,1
1,1,1,740.0,123.333333,250.0,65.0,64.204534,435.0,72.500000,110.0,...,0,9.666667,0,6,15,0.28000,0.600000,-0.320000,0,0
2,2,1,745.0,124.166667,250.0,60.0,64.382753,505.0,84.166667,130.0,...,1,9.750000,0,2,13,0.00000,0.240000,-0.240000,-1,0
3,3,1,730.0,121.666667,250.0,60.0,65.362239,465.0,77.500000,110.0,...,-1,5.750000,0,2,10,0.13000,0.540000,-0.060000,2,0
4,4,1,685.0,114.166667,250.0,50.0,70.794107,455.0,75.833333,110.0,...,0,15.200000,0,5,13,-0.03000,-0.180000,0.320000,-1,0


# Preprocessing

In [3]:
# Preprocessing - LightGBM gestisce bene valori raw (no scaling necessario)
exclude_cols = ['battle_id', 'player_won']
string_cols = train_df.select_dtypes(include=['object']).columns.tolist()
exclude_cols.extend(string_cols)

ALL_NUMERIC_FEATURES = [c for c in train_df.columns if c not in exclude_cols]
FEATURES = ALL_NUMERIC_FEATURES

print(f'Num FEATURES numeriche: {len(FEATURES)}')

# Imputazione mediana
num_df = train_df[FEATURES].astype(float).replace([np.inf, -np.inf], np.nan)
medians = num_df.median()
train_imputed = num_df.fillna(medians)
train_preproc_df = train_imputed.copy()

y = train_df['player_won'].astype(int).values
X = train_preproc_df.values

print('Preprocessing completato.')
print('Dataset completo size:', X.shape[0])
print('Features:', len(FEATURES))

# Allinea test
test_aligned = test_df.reindex(columns=FEATURES, fill_value=np.nan).astype(float).replace([np.inf, -np.inf], np.nan)
test_imputed = test_aligned.fillna(medians)
test_preproc_df = pd.DataFrame(test_imputed.values, columns=FEATURES, index=test_df.index)

Num FEATURES numeriche: 289
Preprocessing completato.
Dataset completo size: 10000
Features: 289


# Hyperparameter Optimization (Optuna + LightGBM)

In [34]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd # Assicurati che pandas sia importato

# 1. PREPARAZIONE DATI ROBUSTA
# Identifica le colonne da escludere (ID, target, eventuali errori)
cols_to_drop = ['battle_id', 'player_won', 'error']

# Identifica automaticamente le colonne numeriche
# Questo passaggio è fondamentale per evitare l'errore "could not convert string to float"
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

# La lista finale delle feature è: (colonne numeriche) MENO (colonne da escludere)
FEATURES = [c for c in numeric_cols if c not in cols_to_drop]

print(f"Feature numeriche selezionate: {len(FEATURES)}")
# print(f"Esempio feature escluse (non numeriche): {[c for c in train_df.columns if c not in numeric_cols][:5]}")

# Crea le matrici X e y
X = train_df[FEATURES].values
y = train_df['player_won'].astype(int).values

print(f"Optuna ready: X shape {X.shape}, y shape {y.shape}")

# 2. DEFINIZIONE FUNZIONE OBIETTIVO (senza pruning callback per sicurezza)
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 0, 7),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.5),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []

    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        model = lgb.LGBMClassifier(**param)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',
            callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
        )
        preds = model.predict(X_val)
        cv_scores.append(accuracy_score(y_val, preds))

    return np.mean(cv_scores)

# 3. ESECUZIONE STUDIO OPTUNA
# Se esiste già uno studio precedente in memoria, potresti volerlo cancellare o dargli un nuovo nome
try:
    optuna.delete_study(study_name='LGBM_Pokemon_V4', storage=None)
except:
    pass

print("Inizio ottimizzazione Optuna...")
study = optuna.create_study(direction='maximize', study_name='LGBM_Pokemon_V4')

# Esegui per N trial o per un tempo massimo (es. 30 minuti = 1800 secondi)
# 20-30 trial potrebbero bastare per vedere un miglioramento se hai poco tempo
study.optimize(objective, n_trials=150, timeout=7200) 

print("\n✅ Ottimizzazione completata!")
print(f"Miglior Accuracy (CV media): {study.best_value:.4f}")
print("Migliori parametri:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

[I 2025-11-08 19:19:48,978] A new study created in memory with name: LGBM_Pokemon_V4


Feature numeriche selezionate: 289
Optuna ready: X shape (10000, 289), y shape (10000,)
Inizio ottimizzazione Optuna...


[I 2025-11-08 19:19:49,797] Trial 0 finished with value: 0.8384 and parameters: {'learning_rate': 0.18904519102765419, 'n_estimators': 400, 'num_leaves': 50, 'max_depth': 3, 'min_child_samples': 65, 'subsample': 0.8733912768784295, 'subsample_freq': 7, 'colsample_bytree': 0.9149167555916753, 'reg_alpha': 2.421275327166644, 'reg_lambda': 7.131882789133064e-07, 'min_split_gain': 0.3045255421333785}. Best is trial 0 with value: 0.8384.
[I 2025-11-08 19:19:51,805] Trial 1 finished with value: 0.8387 and parameters: {'learning_rate': 0.08961174206814042, 'n_estimators': 1000, 'num_leaves': 133, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.6360961119647561, 'subsample_freq': 5, 'colsample_bytree': 0.8499862058017057, 'reg_alpha': 7.183961014067617e-08, 'reg_lambda': 7.7507482097572105, 'min_split_gain': 0.15687167888162334}. Best is trial 1 with value: 0.8387.
[I 2025-11-08 19:19:56,189] Trial 2 finished with value: 0.8389 and parameters: {'learning_rate': 0.014514817450789967, 'n


✅ Ottimizzazione completata!
Miglior Accuracy (CV media): 0.8430
Migliori parametri:
    learning_rate: 0.013679823261876542
    n_estimators: 650
    num_leaves: 97
    max_depth: 7
    min_child_samples: 70
    subsample: 0.6161988793364829
    subsample_freq: 4
    colsample_bytree: 0.6136973726187104
    reg_alpha: 2.261339389106705e-06
    reg_lambda: 1.8764227095178463
    min_split_gain: 0.3554832229238166


# 10-Fold Cross-Validation

In [4]:
print("=== 10-Fold Cross-Validation - LightGBM ===")
best_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'seed': 42,
    'learning_rate': 0.013679823261876542,
    'n_estimators': 650,
    'num_leaves': 97,
    'max_depth': 7,
    'min_child_samples': 70,
    'subsample': 0.6161988793364829,
    'subsample_freq': 4,
    'colsample_bytree': 0.6136973726187104,
    'reg_alpha': 2.261339389106705e-06,
    'reg_lambda': 1.8764227095178463,
    'min_split_gain': 0.3554832229238166
}

print(f"Parametri utilizzati: {best_params}\n")

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
outer_accuracies = []
train_accuracies = []
train_val_gaps = []
folds_info = []

fold_idx = 0
for train_idx, val_idx in skf.split(X, y):
    fold_idx += 1
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    # Train LightGBM con early stopping
    clf = lgb.LGBMClassifier(**best_params)
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
    )
    
    # Validation predictions
    y_pred_val = clf.predict(X_val)
    y_proba_val = clf.predict_proba(X_val)[:, 1]
    val_acc = accuracy_score(y_val, y_pred_val)
    outer_accuracies.append(val_acc)
    
    # Train accuracy
    y_pred_tr = clf.predict(X_tr)
    tr_acc = accuracy_score(y_tr, y_pred_tr)
    gap = tr_acc - val_acc
    train_accuracies.append(tr_acc)
    train_val_gaps.append(gap)
    
    # Best iteration info
    best_iter = clf.best_iteration_ if hasattr(clf, 'best_iteration_') else clf.n_estimators
    
    folds_info.append({
        'fold': fold_idx,
        'val_acc': float(val_acc),
        'train_acc': float(tr_acc),
        'gap': float(gap),
        'best_iteration': int(best_iter),
        'val_idx': val_idx,
        'y_true': y_val.astype(int),
        'y_pred': y_pred_val.astype(int),
        'y_proba': y_proba_val.astype(float)
    })
    
    print(f'Fold {fold_idx}: train={len(y_tr)}, val={len(y_val)}, '
          f'val_acc={val_acc*100:.2f}%, train_acc={tr_acc*100:.2f}%, '
          f'gap={gap*100:.2f}%, best_iter={best_iter}')

print('\n' + '='*60)
print('Risultati Cross-Validation')
print('='*60)
for i, info in enumerate(folds_info, 1):
    print(f"  Fold {i}: val_acc={info['val_acc']*100:.2f}%, "
          f"train_acc={info['train_acc']*100:.2f}%, gap={info['gap']*100:.2f}%")

print(f'\nMean CV accuracy: {np.mean(outer_accuracies)*100:.2f}%')
print(f'Std CV accuracy:  {np.std(outer_accuracies)*100:.2f}%')
print(f'Mean train accuracy: {np.mean(train_accuracies)*100:.2f}%')
print(f'Mean gap (train - val): {np.mean(train_val_gaps)*100:.2f}%')
print(f'Min/Max val acc:  {np.min(outer_accuracies)*100:.2f}% / {np.max(outer_accuracies)*100:.2f}%')

worst_idx = int(np.argmin(outer_accuracies))
print(f"\nPeggiore fold: #{worst_idx+1} con val_acc={outer_accuracies[worst_idx]*100:.2f}%")

=== 10-Fold Cross-Validation - LightGBM ===
Parametri utilizzati: {'objective': 'binary', 'metric': 'binary_logloss', 'boosting_type': 'gbdt', 'verbosity': -1, 'seed': 42, 'learning_rate': 0.013679823261876542, 'n_estimators': 650, 'num_leaves': 97, 'max_depth': 7, 'min_child_samples': 70, 'subsample': 0.6161988793364829, 'subsample_freq': 4, 'colsample_bytree': 0.6136973726187104, 'reg_alpha': 2.261339389106705e-06, 'reg_lambda': 1.8764227095178463, 'min_split_gain': 0.3554832229238166}

Fold 1: train=9000, val=1000, val_acc=83.20%, train_acc=87.31%, gap=4.11%, best_iter=311
Fold 2: train=9000, val=1000, val_acc=84.60%, train_acc=89.83%, gap=5.23%, best_iter=579
Fold 3: train=9000, val=1000, val_acc=84.10%, train_acc=89.99%, gap=5.89%, best_iter=582
Fold 4: train=9000, val=1000, val_acc=85.00%, train_acc=89.99%, gap=4.99%, best_iter=612
Fold 5: train=9000, val=1000, val_acc=85.10%, train_acc=89.24%, gap=4.14%, best_iter=502
Fold 6: train=9000, val=1000, val_acc=83.60%, train_acc=89.06

# Make Submission

In [39]:
print("=== Submission con LightGBM trainato su tutto il dataset ===")

# Train finale su tutto il dataset
submission_model = lgb.LGBMClassifier(**best_params)
submission_model.fit(X, y)

# Predict su test
X_test_matrix = test_preproc_df.values
test_predictions = submission_model.predict(X_test_matrix).astype(int)

# Crea submission
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'].astype(np.int64),
    'player_won': test_predictions.astype(np.int64)
})

submission_path = 'submission_lightgbm.csv'
submission_df.to_csv(submission_path, index=False)

print(f"✅ File di submission salvato in {submission_path}")
print(f"Modello: LightGBM trainato su {len(X)} samples")
print(f"Stima CV accuracy: {np.mean(outer_accuracies)*100:.2f}% ± {np.std(outer_accuracies)*100:.2f}%")
print(f"Mean gap: {np.mean(train_val_gaps)*100:.2f}%")
print("\nPreview submission:")
display(submission_df.head(10))
print(f"\nDistribuzione predizioni: {submission_df['player_won'].value_counts().to_dict()}")

=== Submission con LightGBM trainato su tutto il dataset ===
✅ File di submission salvato in submission_lightgbm.csv
Modello: LightGBM trainato su 10000 samples
Stima CV accuracy: 84.11% ± 0.74%
Mean gap: 5.05%

Preview submission:


,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1



Distribuzione predizioni: {0: 2542, 1: 2458}
